<a href="https://colab.research.google.com/github/Prashanna-Raj-Pandit/NLP/blob/main/Introduction_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi -L

/bin/bash: line 1: nvidia-smi: command not found


# Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/Prashanna-Raj-Pandit/Transfer-Learning--TensorFlow/refs/heads/main/helper_functions.py

--2025-05-20 04:22:37--  https://raw.githubusercontent.com/Prashanna-Raj-Pandit/Transfer-Learning--TensorFlow/refs/heads/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10129 (9.9K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   9.89K  --.-KB/s    in 0s      

2025-05-20 04:22:38 (61.3 MB/s) - ‘helper_functions.py’ saved [10129/10129]



In [3]:
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves, compare_historys

## Get a text dataset

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data("nlp_getting_started.zip")

--2025-05-20 04:22:55--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.98.207, 74.125.135.207, 173.194.202.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.98.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2025-05-20 04:22:56 (108 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing data
with the help of pandas


In [5]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# suffling train data
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
train_df.target.value_counts()

,count
target,
0,4342
1,3271


In [8]:
# how many samples?
len(train_df),len(test_df)

(7613, 3263)

In [9]:
# Lets visualize some random samples.
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"Target:{target}","real disaster" if target>0 else "not real disaster")
  print(f"Text: {text}")
  print("_____")

Target:0 not real disaster
Text: 12' 72W CREE LED Work Light Bar Alloy Spot Flood Combo Diving Offroad 4WD Boat - Full readÛ_ http://t.co/MJMwA72ER6 http://t.co/ADx9iYi246
_____
Target:1 real disaster
Text: FedEx stops shipping potential bioterror pathogens http://t.co/tkeOAeDQKq #trucking
_____
Target:0 not real disaster
Text: How Empire Avenue crushed my soul http://t.co/X9OFV1kMv7 via @markwschaefer
_____
Target:0 not real disaster
Text: @FEVWarrior -with the screeching siren accompanying it just before he walked out.
_____
Target:0 not real disaster
Text: Whirlwind Head Scissor on @alexhammerstone @kttape ktfounder #RemyMarcel #FroFroFroÛ_ https://t.co/B19z8Vi3td
_____


In [10]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


## split data in to training and validation set

In [11]:
from sklearn.model_selection import train_test_split
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size=0.1, # use 10% of training data for validation
                                                                       random_state=42)

In [12]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [13]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Text vectorizarion (Tokenization)
### Converting text into numbers

*   Tokenization: Tokenization is the process of breaking down text into smaller units called tokens, which can be words, subwords, or even characters. Its a direct mapping of token to a number
*   Embedding: Create a matrix of feature vector for each token.
Embeddings convert tokens (words/subwords) into numerical vectors so that machines can process them. They capture semantic meaning (e.g., "king" is closer to "queen" than to "apple").Contextual Embeddings (Modern NLP)

Words get different vectors based on context (e.g., BERT, GPT).



In [14]:
import tensorflow as tf
text_vectorizer=tf.keras.layers.TextVectorization(max_tokens=None,
                                                  standardize='lower_and_strip_punctuation',
                                                  split='whitespace',
                                                  ngrams=None, #create groups of n-words
                                                  output_mode='int',
                                                  output_sequence_length=None, # how long do u want your sequence to be?
                                                  pad_to_max_tokens=False
                                                  )


### find the average number of tokens or words in training samples

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
# setup text vectorization variables
max_vocab_length=10000 # Maximum number of words to have in our vocabolary
max_length=15 #max length our sentences will be
text_vectorizer=tf.keras.layers.TextVectorization(max_tokens=max_vocab_length,
                                                  output_mode='int',
                                                  output_sequence_length=max_length)

In [17]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [18]:
# lets create a sample sentences and tokenize it
sample_sentence="The recent earthquake has destroyed the entire city"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2, 2756,  290,   41,  351,    2,  855,  182,    0,    0,    0,
           0,    0,    0,    0]])>

In [19]:
# selecting a random sample from the train dataset and tokenize it
import random
random_sentence=random.choice(train_sentences)
print(f"Random Sample: {random_sentence}")
print(f"Vectorized sample: {text_vectorizer([random_sentence])}")

Random Sample: I was body Bagging on the ????Today...4got to post my score http://t.co/81g18wSAUk
Vectorized sample: [[   8   23   83  490   11    2 7518    5  371   13 2716    1    0    0
     0]]


In [20]:
# Get the unique words in the vocabulary
words_in_vocab=text_vectorizer.get_vocabulary() # Get all the unique words in the training dataset
top_5_words=words_in_vocab[:5] # get the top 5 most common words
bottom_5_words=words_in_vocab[-5:] # get the least 5 common words
print(f"Words in Vocab: {len(words_in_vocab)}")
print(f"Top 5 words: {top_5_words}")
print(f"Least 5 words: {bottom_5_words}")

Words in Vocab: 10000
Top 5 words: ['', '[UNK]', np.str_('the'), np.str_('a'), np.str_('in')]
Least 5 words: [np.str_('pages'), np.str_('paeds'), np.str_('pads'), np.str_('padres'), np.str_('paddytomlinson1')]


### creating an Embedding using Embedding layer

*   input_dim=the size of our vocabolary
*   output_dim= the size of the output embedding vector
*   input_length= size of the sequences being passed to the embedding layer

Key Properties of an Embedding Layer

**Trainable:**

* The vectors are updated via backpropagation (like other weights in a neural network).

* Pretrained embeddings (e.g., GloVe) can also be fine-tuned.

**Efficiency:**
More efficient than one-hot encoding + dense layers (avokes high-dimensional sparse matrices).

**Interpretability:**
Learned embeddings often capture semantic relationships (e.g., similar words cluster together).


In [21]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=max_vocab_length, # set input shape
                           output_dim=128, # output shape
                           input_length=max_length # how long is each input
                           )

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [22]:
# embed the random sample. i.e turn into dense vector of fixed size
print("Random sample: ",random_sentence)
print("After Embedding:...................")
sample_embed=embedding(text_vectorizer([random_sentence]))
sample_embed

Random sample:  I was body Bagging on the ????Today...4got to post my score http://t.co/81g18wSAUk
After Embedding:...................


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00908017, -0.02939413,  0.01804289, ...,  0.04106123,
          0.029697  ,  0.01512781],
        [-0.04577357, -0.00506413,  0.03367622, ...,  0.03213085,
          0.00159549,  0.03636396],
        [ 0.0010713 , -0.02350105, -0.03488918, ..., -0.03865815,
         -0.01560415, -0.01136728],
        ...,
        [-0.00834791, -0.0463247 , -0.01742047, ...,  0.01591024,
          0.01177416, -0.04401244],
        [-0.00834791, -0.0463247 , -0.01742047, ...,  0.01591024,
          0.01177416, -0.04401244],
        [-0.00834791, -0.0463247 , -0.01742047, ...,  0.01591024,
          0.01177416, -0.04401244]]], dtype=float32)>

In [23]:
# checkout single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00908017, -0.02939413,  0.01804289,  0.02909129, -0.04290458,
        -0.00390773, -0.00729876,  0.02507896,  0.02291006,  0.03181798,
         0.04877057, -0.04568641,  0.02994558, -0.0426631 , -0.0119402 ,
        -0.02276023, -0.02935555, -0.02285813, -0.03221103,  0.01399336,
        -0.02197857,  0.03875719, -0.01593249,  0.04498807, -0.00295243,
        -0.03896997, -0.01713477, -0.04630613,  0.01622535, -0.02838467,
        -0.02884306,  0.00663877, -0.04280554, -0.02753587,  0.01583172,
         0.0201797 ,  0.02990954,  0.02464366,  0.01447303,  0.04937741,
         0.00271485,  0.008952  , -0.01582638,  0.03655225, -0.02130699,
        -0.04362921, -0.00460882,  0.03346865, -0.02086365, -0.04700242,
        -0.00103756,  0.02075544,  0.02396563,  0.01291596,  0.00775719,
        -0.02413945, -0.04715599,  0.0364807 , -0.01691016, -0.03596952,
         0.0083501 ,  0.0403628 ,  0.04014405,  0.02202534,  0.00943452,
  

This is the embedding for the first word of sentence

# Model 0: creating a baseline
To create baseline, let's use Sklearn Multimodal Naive Bayes using TF-IDF formula to convert text into numbers.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
# create tokenization and modeling pipeline
model_0=Pipeline([ # pipeline is like keras Sequantial here
    ('tfidf',TfidfVectorizer()), # convert words into numbers using tfidf
    ('clf',MultinomialNB()), #model the text
])
# Fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

## Evaluate baseline model

In [25]:
baseline_accuracy=model_0.score(val_sentences,val_labels)
print(f"Model 0 (Baseline) Accuracy: {baseline_accuracy*100:.3f} %")

Model 0 (Baseline) Accuracy: 79.265 %


In [26]:
baseline_preds=model_0.predict(val_sentences)
print("Predicted labels")
baseline_preds[:20]

Predicted labels


array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [27]:
print("Actual labels")
val_labels[:20]

Actual labels


array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [28]:
def evaluate_model(model,true_labels,predicted_labels):
  precision=tf.keras.metrics.Precision()
  precision.update_state(true_labels,predicted_labels)
  recall=tf.keras.metrics.Recall()
  recall.update_state(true_labels,predicted_labels)
  f1=(2*(precision.result()*recall.result()))/(precision.result()+recall.result())
  print(f"Precision: {precision.result()*100:.2f}%")
  print(f"Recall: {recall.result()*100:.2f}%")
  print(f"F1 score: {f1*100:.2f}%")

evaluate_model(model_0,val_labels,baseline_preds)



Precision: 88.62%
Recall: 62.64%
F1 score: 73.40%


In [29]:
# from sk-learn
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_performance_metrics(true_labels,predicted_labels):
  accuracy=accuracy_score(true_labels,predicted_labels)
  precision_score,recall_score,f1_score,_=precision_recall_fscore_support(true_labels,predicted_labels,average="weighted")
  model_results={
      "accuracy":accuracy,
      "precision":precision_score,
      "recall":recall_score,
      "f1_score":f1_score
  }
  return model_results

In [30]:
model_0_performance=calculate_performance_metrics(val_labels,baseline_preds)
model_0_performance

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

The two evaluation methods (evaluate_model and calculate_performance_metrics) produce different results because:
* Different Libraries & Implementations

tf.keras.metrics (TensorFlow) and sklearn.metrics (scikit-learn) compute metrics differently.

Example:

>tf.keras.metrics.Precision() uses micro-averaging by default.

>sklearn.metrics.precision_recall_fscore_support() uses weighted averaging (as specified by average="weighted").
Averaging Method

* Weighted vs. Micro Averaging:

>Weighted (scikit-learn): Accounts for class imbalance by weighting metrics by class support.

>Micro (TensorFlow): Aggregates all classes equally, which can skew results if classes are imbalanced.

# Model 1: Simple dense model

In [31]:
# Create a tensorboard callback
from helper_functions import create_tensorboard_callback
# create a directory to save the tensorboard logs
SAVE_DIR="mdoel_logs"

In [32]:
# Building a model with functional API
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string") # inputs are one dimentional string
x=text_vectorizer(inputs) # turn input text into number
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x) # lower the dimentionality of the embedding
outputs=layers.Dense(1,activation="sigmoid")(x)
model_1=tf.keras.Model(inputs,outputs,name="model_1_dense")

In [33]:
model_1.summary()

Model: "model_1_dense"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,280,129 (4.88 MB)

 Trainable params: 1,280,129 (4.88 MB)

 Non-trainable params: 0 (0.00 B)

In [34]:
#compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1_history=model_1.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                          callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="model_1_dense")])


Saving TensorBoard log files to: mdoel_logs/model_1_dense/20250520-042259
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - accuracy: 0.6453 - loss: 0.6469 - val_accuracy: 0.7507 - val_loss: 0.5346
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8151 - loss: 0.4625 - val_accuracy: 0.7730 - val_loss: 0.4837
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.8568 - loss: 0.3565 - val_accuracy: 0.7913 - val_loss: 0.4592
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.8904 - loss: 0.2925 - val_accuracy: 0.7913 - val_loss: 0.4668
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9127 - loss: 0.2407 - val_accuracy: 0.7835 - val_loss: 0.4828


In [35]:
model_1.evaluate(val_sentences,val_labels)

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7799 - loss: 0.5170 


[0.4828168451786041, 0.7834645509719849]

In [36]:
# make predictions
model_1_pred_probability=model_1.predict(val_sentences)
model_1_pred_probability.shape

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


(762, 1)

In [37]:
model_1_pred_probability[0]

array([0.34013173], dtype=float32)

In [38]:
model_1_pred_probability=tf.squeeze(tf.round(model_1_pred_probability))
model_1_pred_probability[:50]

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1.],
      dtype=float32)>

In [39]:
val_labels[:50]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0])

In [40]:
model_1_perfornamce=calculate_performance_metrics(true_labels=val_labels,predicted_labels=model_1_pred_probability)
model_1_perfornamce

{'accuracy': 0.7834645669291339,
 'precision': 0.7898206272603021,
 'recall': 0.7834645669291339,
 'f1_score': 0.7798782237240588}

In [41]:
# compare with the baseline
model_0_performance

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

Here, Our base line model outperforms the Deep Learning Model 1

# Visualizing embedding layer

In [42]:
len(words_in_vocab), words_in_vocab[:10] # the most common words

(10000,
 ['',
  '[UNK]',
  np.str_('the'),
  np.str_('a'),
  np.str_('in'),
  np.str_('to'),
  np.str_('of'),
  np.str_('and'),
  np.str_('i'),
  np.str_('is')])

In [43]:
# get the weight matrix of embedding layer
embeded_weights=model_1.get_layer("embedding").get_weights()[0]
embeded_weights

array([[-0.01636451, -0.06510636, -0.00174436, ...,  0.03380989,
        -0.00761036, -0.02650117],
       [ 0.00389651, -0.05746577,  0.03086385, ...,  0.03602378,
        -0.05304493,  0.03736534],
       [-0.00399978, -0.05715721,  0.05204615, ...,  0.01283794,
         0.01168613,  0.03959261],
       ...,
       [-0.01663307, -0.03065559, -0.00249141, ..., -0.02230046,
         0.04757012,  0.02260187],
       [-0.04235464, -0.00923517,  0.07123073, ...,  0.02922026,
        -0.03006231,  0.05601089],
       [-0.06625894, -0.11577218,  0.03489673, ...,  0.04054838,
        -0.11534824,  0.09219477]], dtype=float32)

In [44]:
embeded_weights.shape

(10000, 128)

## Embedding Projector

In [45]:
# Write the weights to disk. To use the Embedding Projector, you will upload two files in tab separated format:
# a file of vectors (containing the embedding), and a file of meta data (containing the words).
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embeded_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [46]:
# If you are running this tutorial in Colaboratory, you can use the following snippet to download these files to your local machine
#  (or use the file browser, View -> Table of contents -> File browser).
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Model: 2
LSTM : Long short term memory
> Input->Tokenizer->Embeddings->Layers(RNN/Dense)->Output(label probability)

In [47]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),name="input_layer",dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
print(x.shape)
# x=layers.LSTM(units=64,return_sequences=True)(x) # when we are stacking the LSTM cells we need to set the return_sequences= true
print(x.shape)
x=layers.LSTM(64)(x)
print(x.shape)
# x=layers.Dense(64,activation="relu")(x)
outputs=layers.Dense(1,activation="sigmoid")(x)

model_2=tf.keras.Model(inputs,outputs,name="model_2_LSTM")

(None, 15, 128)
(None, 15, 128)
(None, 64)


In [48]:
model_2.summary()

Model: "model_2_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,329,473 (5.07 MB)

 Trainable params: 1,329,473 (5.07 MB)

 Non-trainable params: 0 (0.00 B)

In [49]:
# compile the mdoel
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [50]:
# fit the model
model_2_history=model_2.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="model_2_lstm")])

Saving TensorBoard log files to: mdoel_logs/model_2_lstm/20250520-042320
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 8s 23ms/step - accuracy: 0.8875 - loss: 0.2985 - val_accuracy: 0.7743 - val_loss: 0.5325
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9426 - loss: 0.1608 - val_accuracy: 0.7717 - val_loss: 0.6402
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.9526 - loss: 0.1209 - val_accuracy: 0.7822 - val_loss: 0.7017
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.9607 - loss: 0.1046 - val_accuracy: 0.7743 - val_loss: 0.7086
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.9650 - loss: 0.0967 - val_accuracy: 0.7887 - val_loss: 0.9991


In [51]:
# make prediction
model_2_pred=model_2.predict(val_sentences)
model_2_pred[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


array([[1.8542568e-01],
       [9.3830371e-01],
       [9.9985886e-01],
       [1.2263768e-01],
       [7.9907884e-04],
       [9.9971104e-01],
       [9.6725339e-01],
       [9.9990278e-01],
       [9.9986053e-01],
       [7.5994986e-01]], dtype=float32)

In [52]:
model_2_pred_probability=tf.squeeze(tf.round(model_2_pred))
model_2_pred_probability[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

## Model 2 vs Model 1 vs Model 0

Still our baseline beats our Model 1 and Model 2

In [53]:
model_2_performance=calculate_performance_metrics(true_labels=val_labels,predicted_labels=model_2_pred_probability)
model_2_performance

{'accuracy': 0.7887139107611548,
 'precision': 0.7884679207859479,
 'recall': 0.7887139107611548,
 'f1_score': 0.78819430801797}

In [54]:
model_1_perfornamce

{'accuracy': 0.7834645669291339,
 'precision': 0.7898206272603021,
 'recall': 0.7834645669291339,
 'f1_score': 0.7798782237240588}

In [55]:
model_0_performance

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

# Model 3: GRU
The GRU cell has similar features to an LSTM but has less parameters

In [56]:
# Build an RNN using GRU cell
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GRU(64)(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_3=tf.keras.Model(inputs,outputs,name="model_3_GRU")

In [57]:
model_3.summary()

Model: "model_3_GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 64)             │        37,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,317,313 (5.03 MB)

 Trainable params: 1,317,313 (5.03 MB)

 Non-trainable params: 0 (0.00 B)

In [58]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_3_history=model_3.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="model_3_GRU")])

Saving TensorBoard log files to: mdoel_logs/model_3_GRU/20250520-042354
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - accuracy: 0.8560 - loss: 0.2892 - val_accuracy: 0.7808 - val_loss: 0.7732
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9659 - loss: 0.0870 - val_accuracy: 0.7756 - val_loss: 0.9180
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.9660 - loss: 0.0829 - val_accuracy: 0.7769 - val_loss: 0.9367
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - accuracy: 0.9757 - loss: 0.0574 - val_accuracy: 0.7703 - val_loss: 1.1292
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9793 - loss: 0.0456 - val_accuracy: 0.7664 - val_loss: 1.1699


In [59]:
model_3_pred=model_3.predict(val_sentences)
model_3_pred[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step


array([[7.3659647e-04],
       [8.5576504e-01],
       [9.9994582e-01],
       [5.8373086e-02],
       [2.9089791e-04],
       [9.9991393e-01],
       [9.8149163e-01],
       [9.9998432e-01],
       [9.9995792e-01],
       [9.8442626e-01]], dtype=float32)

In [60]:
model_3_pred_probability=tf.squeeze(tf.round(model_3_pred))
model_3_pred_probability[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [61]:
model_3_performance=calculate_performance_metrics(true_labels=val_labels,predicted_labels=model_3_pred_probability)
model_3_performance

{'accuracy': 0.7664041994750657,
 'precision': 0.7681630705252752,
 'recall': 0.7664041994750657,
 'f1_score': 0.764152388105216}

In [62]:
model_0_performance

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1_score': 0.7862189758049549}

# Model 4

Normal RNN's go from the left to right but a bidirectional RNN's goes from the left to right as well as right to left.

In [63]:
from tensorflow.keras import layers
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)
x=embedding(x)
# x=layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x=layers.Bidirectional(layers.LSTM(64))(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_4=tf.keras.Model(inputs,outputs,name="model_4_bidirectional")


In [64]:
# summary
model_4.summary()

Model: "model_4_bidirectional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,378,945 (5.26 MB)

 Trainable params: 1,378,945 (5.26 MB)

 Non-trainable params: 0 (0.00 B)

In [65]:
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [66]:
# fit the model
model_4_history=model_4.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="model 4 bidirectional")])

Saving TensorBoard log files to: mdoel_logs/model 4 bidirectional/20250520-042431
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 12s 37ms/step - accuracy: 0.9519 - loss: 0.1889 - val_accuracy: 0.7756 - val_loss: 0.9780
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 10s 36ms/step - accuracy: 0.9788 - loss: 0.0493 - val_accuracy: 0.7664 - val_loss: 1.1191
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 14s 54ms/step - accuracy: 0.9776 - loss: 0.0442 - val_accuracy: 0.7690 - val_loss: 1.3820
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 18s 44ms/step - accuracy: 0.9804 - loss: 0.0448 - val_accuracy: 0.7717 - val_loss: 1.4042
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.9841 - loss: 0.0356 - val_accuracy: 0.7717 - val_loss: 1.3519


In [67]:
#make prediction on bodirectional RNN
model_4_pred=model_4.predict(val_sentences)
model_4_pred[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step


array([[1.8456042e-03],
       [5.9422904e-01],
       [9.9997276e-01],
       [2.7187312e-01],
       [1.6874759e-05],
       [9.9950254e-01],
       [8.6252338e-01],
       [9.9998569e-01],
       [9.9995524e-01],
       [8.9974713e-01]], dtype=float32)

In [68]:
# converting pred prob to the pred labels
model_4_preds=tf.squeeze(tf.round(model_4_pred))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [69]:
# calculating the metrics
model_4_performance=calculate_performance_metrics(true_labels=val_labels,predicted_labels=model_4_preds)
model_4_performance

{'accuracy': 0.7716535433070866,
 'precision': 0.7741380916586217,
 'recall': 0.7716535433070866,
 'f1_score': 0.7691811868378113}

# CNN for the text.
we have used CNN for the images which is 2D but our text data is 1D

Inputs-> Tokenozation-> Embedding-> layer(s) (typically conv1D or pooling layers) -> outputs (class probalilities)

# Model 5 : Conv1D

In [70]:
from tensorflow.keras import layers
embedding_test=embedding(text_vectorizer(["hi there, how are you"]))
conv1d=layers.Conv1D(filters=32,kernel_size=5,padding="valid",activation="relu")
conv1d_output=conv1d(embedding_test)
max_pool=layers.GlobalAveragePooling1D()
max_pool_output=max_pool(conv1d_output)
max_pool_output=max_pool(conv1d_output)
embedding_test.shape,conv1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [71]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.05748843,  0.00953171, -0.03204268, ...,  0.0172586 ,
         -0.01893646,  0.08792068],
        [-0.07327786, -0.09062102,  0.03925657, ...,  0.05581751,
         -0.10322063,  0.07682306],
        [ 0.02288945, -0.02808686,  0.08429533, ...,  0.06248755,
         -0.04360579,  0.0348763 ],
        ...,
        [ 0.01595721, -0.04129804, -0.0123491 , ...,  0.00871723,
          0.00061557, -0.05890075],
        [ 0.01595721, -0.04129804, -0.0123491 , ...,  0.00871723,
          0.00061557, -0.05890075],
        [ 0.01595721, -0.04129804, -0.0123491 , ...,  0.00871723,
          0.00061557, -0.05890075]]], dtype=float32)>

In [76]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.Conv1D(filters=64,kernel_size=5, strides=1, activation="relu", padding="valid")(x)
x=layers.GlobalMaxPool1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_5=tf.keras.Model(inputs,outputs,name='model_5_conv1d')
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_5.summary()

Model: "model_5_conv1d"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 1)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ text_vectorization_1            │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 11, 64)         │        41,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_4          │ (None, 64)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,321,089 (5.04 MB)

 Trainable params: 1,321,089 (5.04 MB)

 Non-trainable params: 0 (0.00 B)

In [77]:
model_5_history= model_5.fit(train_sentences,
                             train_labels,
                             epochs=5,
                             validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,experiment_name="Conv1D")]
                            )

Saving TensorBoard log files to: mdoel_logs/Conv1D/20250520-044137
Epoch 1/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.9465 - loss: 0.1793 - val_accuracy: 0.7717 - val_loss: 0.9281
Epoch 2/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 19ms/step - accuracy: 0.9717 - loss: 0.0746 - val_accuracy: 0.7677 - val_loss: 1.0311
Epoch 3/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.9785 - loss: 0.0571 - val_accuracy: 0.7638 - val_loss: 1.1289
Epoch 4/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9813 - loss: 0.0483 - val_accuracy: 0.7703 - val_loss: 1.1454
Epoch 5/5
215/215 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.9799 - loss: 0.0488 - val_accuracy: 0.7559 - val_loss: 1.2133


In [80]:
model_5_pred=model_5.predict(val_sentences)
model_5_pred[:10]

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[2.1358810e-01],
       [7.4325305e-01],
       [9.9992913e-01],
       [3.7949245e-02],
       [9.6599813e-08],
       [9.9638391e-01],
       [9.8202157e-01],
       [9.9995399e-01],
       [9.9999964e-01],
       [9.4014031e-01]], dtype=float32)

In [81]:
model_5_pred_probability=tf.squeeze(tf.round(model_5_pred))
model_5_pred_probability[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [82]:
model_5_performance=calculate_performance_metrics(val_labels,model_5_pred_probability)
model_5_performance


{'accuracy': 0.7559055118110236,
 'precision': 0.7558252056840676,
 'recall': 0.7559055118110236,
 'f1_score': 0.7546752743603138}

In [4]:
# We can use this encoding layer in place of our text_vectorizer and embedding layer
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE") #Universal sentense encoder

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

# Load the USE KerasLayer once
use_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                           input_shape=[],
                           dtype=tf.string,
                           trainable=False,
                           name="USE_embedding")

# Functional API model using USE
input_text = tf.keras.Input(shape=(), dtype=tf.string, name="input_text")
embedding = use_layer(input_text)  # directly apply the KerasLayer
x = layers.Dense(64, activation="relu")(embedding)
output = layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs=input_text, outputs=output, name="model_USE")

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Show model summary
model.summary()


KeyboardInterrupt: 